## Lib

In [ ]:
import keras

## Vars

In [ ]:
model_path = model_path = "../../storage/models/generated_model_efficientnetv2_21k_finetuned_1k_v1_trained.keras"
test_dataset_path = "../../storage/datas/tf_datasets/test_dataset"

## Load Model & test Dataset

In [ ]:
# Load model
model = keras.saving.load_model(model_path)

# Load datasets
test_datas = tf.data.experimental.load(test_dataset_path)

## Evaluate

In [ ]:
test_evaluation = model.evaluate(test_datas)
print("Test evaluation:", test_results)